# Open a new beer garden

###  The metology
1) Create a dataframe that holds the districts of Munich with latitudes and longitudes  
2) Use Foursquare to find the top venues in every district  
3) Use one hot encoding to determine the concentrations of each venue category  
4) Slice and sort dataframe to find district with highest concentration of Turkish Restaurants  

### Dataframe of munich

In [6]:
#Imports
import pandas as pd 
import numpy as np
import requests

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

In [3]:
districts_of_munich = ['Altstadt-Lehel', 'Ludwigsvorstadt-Isarvorstadt', 'Maxvorstadt', 'Schwabing-West', 'Au-Haidhausen', 'Sendling',
                       'Sendling-Westpark', 'Schwanthalerhöhe', 'Neuhausen-Nymphenburg', 'Moosach', 'Milbertshofen-Am Hart', 'Schwabing-Freimann',
                       'Bogenhausen', 'Berg am Laim', 'Trudering-Riem', 'Ramersdorf-Perlach', 'Obergiesing-Fasangarten', 'Untergiesing-Harlaching',
                       'Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln', 'Hadern', 'Pasing-Obermenzing', 'Aubing-Lochhausen-Langwied', 
                       'Allach-Untermenzing', 'Feldmoching-Hasenbergl', 'Laim']

In [4]:
districts_df = pd.DataFrame(data = districts_of_munich, columns=['District'])
districts_df.head()

,District
0,Altstadt-Lehel
1,Ludwigsvorstadt-Isarvorstadt
2,Maxvorstadt
3,Schwabing-West
4,Au-Haidhausen


### Using GeoPy to get location

In [7]:
latitudes = []
longitudes = []

In [9]:
for hood in districts_of_munich:
    address = '{}, Munich, Germany'.format(hood)
    
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    
    latitude = location.latitude
    latitudes.append(latitude)
    
    longitude = location.longitude
    longitudes.append(longitude)

In [10]:
districts_df['Latitudes'] = latitudes
districts_df['Longitudes'] = longitudes

In [11]:
districts_df.head()

,District,Latitudes,Longitudes
0,Altstadt-Lehel,48.137828,11.574582
1,Ludwigsvorstadt-Isarvorstadt,48.131771,11.555809
2,Maxvorstadt,48.146570,11.571445
3,Schwabing-West,48.164417,11.570364
4,Au-Haidhausen,48.128753,11.590536


### Top venues of every district using Foursquare

In [19]:
CLIENT_ID = 'XXX' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' )
print('CLIENT_SECRET:')

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius = 2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
munich_venues = getNearbyVenues(names = districts_df['District'],
                                   latitudes = districts_df['Latitudes'],
                                   longitudes = districts_df['Longitudes']
                                  )

In [18]:
munich_venues.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt-Lehel,48.137828,11.574582,Marienplatz,48.137177,11.575561,Plaza
1,Altstadt-Lehel,48.137828,11.574582,Alois Dallmayr,48.138554,11.576750,Gourmet Shop
2,Altstadt-Lehel,48.137828,11.574582,Fischbrunnen,48.137211,11.576047,Fountain
3,Altstadt-Lehel,48.137828,11.574582,St. Peter,48.136530,11.575615,Church
4,Altstadt-Lehel,48.137828,11.574582,Rindermarkt,48.136099,11.573915,Plaza


### one hot encoding

In [21]:
munich_onehot = pd.get_dummies(munich_venues[['Venue Category']], prefix="", prefix_sep="")
munich_onehot['District'] = munich_venues['District']

In [22]:
munich_grouped = munich_onehot.groupby('District').mean().reset_index()
munich_grouped.head()

,District,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Allach-Untermenzing,0.000000,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.000000,...,0.000000,0.0,0.037037,0.0,0.00,0.0,0.0,0.00,0.0,0.0
1,Altstadt-Lehel,0.000000,0.00,0.0,0.01,0.0,0.0,0.01,0.01,0.000000,...,0.000000,0.0,0.000000,0.0,0.01,0.0,0.0,0.01,0.0,0.0
2,Au-Haidhausen,0.000000,0.01,0.0,0.00,0.0,0.0,0.00,0.00,0.000000,...,0.010000,0.0,0.000000,0.0,0.02,0.0,0.0,0.01,0.0,0.0
3,Aubing-Lochhausen-Langwied,0.000000,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.000000,...,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.0
4,Berg am Laim,0.011494,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.045977,...,0.011494,0.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.0


### find dirstrict with highest concentration of beer gardens

In [26]:
new_df = munich_grouped[['District','Beer Garden']]
new_df.sort_values(by=['Beer Garden'], ascending = False).head()

,District,Beer Garden
5,Bogenhausen,0.032609
11,Milbertshofen-Am Hart,0.030000
2,Au-Haidhausen,0.030000
22,Thalkirchen-Obersendling-Forstenried-Fürstenri...,0.030000
17,Schwabing-Freimann,0.030000


### district with most beer gardens

The district with the most beer gardens is Bogenhausen. Therefore circumstances for beer gardens should be very good there.

In [29]:
new_df.sort_values(by=['Beer Garden'], ascending = True).head(10)

,District,Beer Garden
0,Allach-Untermenzing,0.00
3,Aubing-Lochhausen-Langwied,0.00
4,Berg am Laim,0.00
18,Schwabing-West,0.00
6,Feldmoching-Hasenbergl,0.00
7,Hadern,0.00
10,Maxvorstadt,0.00
16,Ramersdorf-Perlach,0.00
15,Pasing-Obermenzing,0.00
1,Altstadt-Lehel,0.01


### district with no beer garden

According to foursquare the first 9 districts above should have no beer gardens. Therefore competition should be very low there.